In [3]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.float32}, 
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Arrrr, shiver me timbers! Me name be Captain Chatbot, the scurviest pirate to ever sail the Seven Seas! Me be a swashbucklin' chatbot, here to regale ye with tales of me adventures on the high seas, and answer yer questions to the best o' me ability. So hoist the colors, me hearty, and let's set sail fer a treasure trove o' conversation!


In [5]:
messages = [
    {"role": "system", "content": "you are a professional translator who translates phrases from Spanish to English on demand. You only answere the translation without any greetings and farewells."},
    {"role": "user", "content": "me llamo iñaki y mi comida favorita son los arroces de marisco"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.1,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


My name is Iñaki and my favorite food is seafood paella.


In [6]:
print(outputs[0]["generated_text"][len(prompt):])

My name is Iñaki and my favorite food is seafood paella.


In [17]:
import tqdm

# load dataset train_es_data.pth
datadir = "/home/iñaki/host_data/dataset_oppositional/"
train_es_dataset_path = datadir + "train_es_data.pth"
train_es_dataset = torch.load(train_es_dataset_path)

# iterate over the dataset and translate the spanish sentences to english
translated = []
for i in tqdm.tqdm(range(len(train_es_dataset[0]))):
    messages = [
        {"role": "system", "content": "you are a professional translator who translates phrases from Spanish to English on demand. You only answere the translation without any greetings and farewells."},
        {"role": "user", "content": train_es_dataset[0][i]},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )
    print(train_es_dataset[0][i])
    print(outputs[0]["generated_text"][len(prompt):])
    print("--------------------")
    translated += [outputs[0]["generated_text"][len(prompt):]]
    break

  0%|          | 0/3600 [08:54<?, ?it/s]

El G7 realiza una simulación de “ Pandemia de Viruela de leopardo ” : los ministros de salud se reúnen “ contra la próxima crisis ” Los ministros de salud del G7 ya están practicando “ la próxima pandemia ” . En Berlín , realizan una simulación del curso de una pandemia de viruela en 2023 , según informa el periódico alemán Bild . Cabe recordar que el mismo escenario futuro , en base esta vez a una modificación de la viruela , viene siendo anunciado por el magnate Bill Gates .   | https :// trikooba . blog / 45408 . html   Síguenos en :   Nuevo TELEGRAM : https :// t . me / trikooba2022   Nuevo FACEBOOK : https :// bit . ly / 38b1CEr INSTAGRAM : instagram . com / trikooba TWITTER : twitter . com / 3Kooba _ com MEWE : bit . ly / 3dxxenE VK : vk . com / trikoobanews 
The G7 is conducting a simulation of a "Leopard Virus Pandemic": health ministers meet "against the next crisis". The G7 health ministers are already practicing "the next pandemic". In Berlin, they are conducting a simulatio

In [20]:
# save new dataset
train_es_translated_data_path = datadir + "train_es-translated_data.pth"
torch.save([translated, train_es_dataset[1]], train_es_translated_data_path)

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)